In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import lxml
import numpy as np
import pandas as pd

immobiliare_url = 'https://www.immobiliare.it/vendita-case/trieste-provincia/?criterio=rilevanza&pag='
real_states_about = []
x = 0
# Iterating pages of the website
def main_immobiliare(req):
    source = req.content
    soup = BeautifulSoup(source, 'lxml')
    real_states = soup.find_all("li", {"class": "nd-list__item in-realEstateResults__item"})
    number_of_real_states = len(real_states)

    def get_real_state_info(real_state):
        less_than_1000 = "⚠ Element not found ⚠"
        calc_per_meter = "⚠ Element not found ⚠"
        price = real_state.contents[0].find('li', {"class": "nd-list__item in-feat__item in-feat__item--main in-realEstateListCard__features--main"})
        if price is not None:
            price = price.text.strip()
        else:
            price = "Element not found."
        space = real_state.find('span', class_='in-mainFeatures__text')
        value_m_squared = space.find_previous_sibling(text=True).strip().replace('€', '').replace(',', '')  # Remove Euro symbol and comma
        if price != "Element not found." and value_m_squared and value_m_squared.isdigit():
            digit_price = price.split()[1]
            new_number = ''.join(filter(str.isdigit, digit_price))  # Keep only digits
            if new_number and new_number.isdigit():
                calc_per_meter = int(new_number) / int(value_m_squared)
                if calc_per_meter < 1000:
                    less_than_1000 = 'True ✅'
                else:
                    less_than_1000 = 'False ❌'
        real_states_about.append({"Price": price, "M**2": value_m_squared, "Status Price per Meter": f"{less_than_1000} The price is {calc_per_meter} per Meter"})

    # Iterating inside the page and applying the function on every real state
    for i in range(number_of_real_states):
        get_real_state_info(real_states[i])

for page in range(0, 57):
    x += 1
    new_immobiliare_url = immobiliare_url + str(x)
    request = requests.get(new_immobiliare_url)
    main_immobiliare(request)


In [3]:
keys = real_states_about[0].keys()
with open("IMMOBILIARE WEB SCRAPING", 'w', encoding='utf-8') as output_file:
        csv_writer = csv.DictWriter(output_file, fieldnames=keys)
        csv_writer.writeheader()
        csv_writer.writerows(real_states_about)
        print("File Created")

File Created


In [4]:
df = pd.DataFrame(real_states_about)

In [5]:
df

,Price,M**2,Status Price per Meter
0,€ 1.600.000,530,False ❌ The price is 3018.867924528302 per Meter
1,€ 497.000,92,False ❌ The price is 5402.173913043478 per Meter
2,€ 850.000,158,False ❌ The price is 5379.7468354430375 per Meter
3,€ 650.000,105,False ❌ The price is 6190.476190476191 per Meter
4,Element not found.,156,⚠ Element not found ⚠ The price is ⚠ Element n...
...,...,...,...
1374,€ 165.000,65,False ❌ The price is 2538.4615384615386 per Meter
1375,€ 295.000,145,False ❌ The price is 2034.4827586206898 per Meter
1376,€ 400.000,800,True ✅ The price is 500.0 per Meter
1377,€ 250.000,110,False ❌ The price is 2272.7272727272725 per Meter


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('IMMOBILIARE WEB SCRAPING')

In [4]:
df.head()

,Price,M**2,Status Price per Meter
0,€ 1.600.000,530.0,False ❌ The price is 3018.867924528302 per Meter
1,€ 497.000,92.0,False ❌ The price is 5402.173913043478 per Meter
2,€ 850.000,158.0,False ❌ The price is 5379.7468354430375 per Meter
3,€ 650.000,105.0,False ❌ The price is 6190.476190476191 per Meter
4,Element not found.,156.0,⚠ Element not found ⚠ The price is ⚠ Element n...


In [17]:
count = 0
for e in df['Status Price per Meter']:
    if e[:4] == 'True':
        count+=1

In [18]:
count

44